## Assistant Agent

AssistantAgent is a built-in agent that uses a language model and has the ability to use tools.

In [4]:
from autogen_ext.models.ollama import OllamaChatCompletionClient

ollama_model_client = OllamaChatCompletionClient(
    model="llama3.2"
)


In [8]:
from autogen_agentchat.agents import AssistantAgent

async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."

async def add_numbers(a : int , b : int) -> int:
    """

    Perform addition of int a and b

    Input :
    
    a : int
    b : int

    Output : 

    a + b 

    """
    return a + b

agentX = AssistantAgent(
    name="assistant",
    model_client=ollama_model_client,
    tools=[web_search , add_numbers],
    system_message="Use tools to solve tasks",
)



In [11]:
from autogen_agentchat.messages import TextMessage                                                                                                      
text_message = TextMessage(content="Find information on AutoGen and perform addition of 3 and 5", source="User")  
text_message

TextMessage(source='User', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 4, 9, 47, 34, 215608, tzinfo=datetime.timezone.utc), content='Find information on AutoGen and perform addition of 3 and 5', type='TextMessage')

In [12]:
# result = await agentX.run(task="Find information on AutoGen and perform addition of 3 and 5")
result = await agentX.run(task=text_message)
print(result.messages[-1].content)

AutoGen is a programming framework for building multi-agent applications.
8


## Streaming Messages

run() : Runs the chat and returns full result at once

run_stream() : Runs the chat and yields results incrementally

In [22]:
async def assistant_run() -> None:
    
    result = await agentX.run(task=text_message)
    print(result.messages[-1].content)

await assistant_run()

AutoGen is a programming framework for building multi-agent applications.
8


In [ ]:
async for chunk in agentX.run_stream(task=text_message):
    print(chunk)
# Use async for with run_stream(), because it’s an async generator, not a coroutine.

source='User' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 4, 9, 47, 34, 215608, tzinfo=datetime.timezone.utc) content='Find information on AutoGen and perform addition of 3 and 5' type='TextMessage'
source='assistant' models_usage=RequestUsage(prompt_tokens=1045, completion_tokens=41) metadata={} created_at=datetime.datetime(2025, 7, 4, 9, 57, 29, 477751, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='12', arguments='{"query": "AutoGen"}', name='web_search'), FunctionCall(id='12', arguments='{"a": "3", "b": "5"}', name='add_numbers')] type='ToolCallRequestEvent'
source='assistant' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 4, 9, 57, 29, 480262, tzinfo=datetime.timezone.utc) content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', name='web_search', call_id='12', is_error=False), FunctionExecutionResult(content='8', name='add_numbers', call_id='12', is_error=False)] typ


**Coroutine**  : 	Runs once and returns a single value	

*await agentX.run()*

**Async Generator** : Yields multiple values over time	

*async for chunk in agentX.run_stream()*

In [20]:
from autogen_agentchat.ui import Console

async def assistant_run_stream(output_stats) -> None:

    await Console(
        agentX.run_stream(task=text_message),
        output_stats=output_stats,  # True : Enable stats printing.
    )

await assistant_run_stream(True)

# Note :  Using Console to print all messages as they appear.
# Note :  Use asyncio.run(assistant_run_stream()) when running in a script.

---------- TextMessage (User) ----------
Find information on AutoGen and perform addition of 3 and 5
---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='10', arguments='{"query": "AutoGen"}', name='web_search'), FunctionCall(id='10', arguments='{"a": "3", "b": "5"}', name='add_numbers')]
[Prompt tokens: 889, Completion tokens: 41]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', name='web_search', call_id='10', is_error=False), FunctionExecutionResult(content='8', name='add_numbers', call_id='10', is_error=False)]
---------- ToolCallSummaryMessage (assistant) ----------
AutoGen is a programming framework for building multi-agent applications.
8
---------- Summary ----------
Number of messages: 4
Finish reason: None
Total prompt tokens: 889
Total completion tokens: 41
Duration: 3.14 seconds


In [17]:
await assistant_run_stream(False)

---------- TextMessage (User) ----------
Find information on AutoGen and perform addition of 3 and 5
---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='7', arguments='{"query": "AutoGen"}', name='web_search'), FunctionCall(id='7', arguments='{"a": "3", "b": "5"}', name='add_numbers')]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', name='web_search', call_id='7', is_error=False), FunctionExecutionResult(content='8', name='add_numbers', call_id='7', is_error=False)]
---------- ToolCallSummaryMessage (assistant) ----------
AutoGen is a programming framework for building multi-agent applications.
8


In [19]:
async def assistant_run_stream() -> None:
    async for message in agentX.run_stream(task=text_message):
        print(message)
        
await assistant_run_stream()


source='User' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 4, 9, 47, 34, 215608, tzinfo=datetime.timezone.utc) content='Find information on AutoGen and perform addition of 3 and 5' type='TextMessage'
source='assistant' models_usage=RequestUsage(prompt_tokens=811, completion_tokens=41) metadata={} created_at=datetime.datetime(2025, 7, 4, 9, 52, 52, 724088, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='9', arguments='{"query": "AutoGen"}', name='web_search'), FunctionCall(id='9', arguments='{"a": "3", "b": "5"}', name='add_numbers')] type='ToolCallRequestEvent'
source='assistant' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 4, 9, 52, 52, 727292, tzinfo=datetime.timezone.utc) content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', name='web_search', call_id='9', is_error=False), FunctionExecutionResult(content='8', name='add_numbers', call_id='9', is_error=False)] type='To

## Structured Output

In [29]:
from typing import Literal
from pydantic import BaseModel


class AgentResponse(BaseModel):
    thoughts: str
    response: Literal["happy", "sad", "neutral"]


agentY = AssistantAgent(
    "assistant" , 
    model_client=ollama_model_client, 
    description="Categorize the input as happy, sad, or neutral following the JSON format.",
    system_message="Solve the task",
    output_content_type=AgentResponse,
)

text_message = TextMessage(content="I am happy." , source="user")

result = await Console(agentY.run_stream(task=text_message))


---------- TextMessage (user) ----------
I am happy.
---------- StructuredMessage[AgentResponse] (assistant) ----------
{"thoughts":"That's wonderful to hear! What's making you happy today?","response":"happy"}


In [32]:
result.messages

[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 4, 10, 7, 30, 30755, tzinfo=datetime.timezone.utc), content='I am happy.', type='TextMessage'),
 StructuredMessage[AgentResponse](source='assistant', models_usage=RequestUsage(prompt_tokens=33, completion_tokens=34), metadata={}, created_at=datetime.datetime(2025, 7, 4, 10, 7, 31, 912790, tzinfo=datetime.timezone.utc), content=AgentResponse(thoughts="That's wonderful to hear! What's making you happy today?", response='happy'), format_string=None, type='StructuredMessage[AgentResponse]')]

In [36]:
print(result.messages[-1])
print(type(result.messages[-1]))

source='assistant' models_usage=RequestUsage(prompt_tokens=33, completion_tokens=34) metadata={} created_at=datetime.datetime(2025, 7, 4, 10, 7, 31, 912790, tzinfo=datetime.timezone.utc) content=AgentResponse(thoughts="That's wonderful to hear! What's making you happy today?", response='happy') format_string=None type='StructuredMessage[AgentResponse]'
<class 'autogen_agentchat.messages.StructuredMessage[AgentResponse]'>


In [38]:
from autogen_agentchat.messages import StructuredMessage

assert isinstance(result.messages[-1], StructuredMessage)
assert isinstance(result.messages[-1].content, AgentResponse)

print("Thought: ", result.messages[-1].content.thoughts)
print("Response: ", result.messages[-1].content.response)


Thought:  That's wonderful to hear! What's making you happy today?
Response:  happy
